In [23]:
# Import required packages (will fail if not available)
import xarray as xr
import pysnptools

print(f"✓ xarray {xr.__version__}")
print(f"✓ pysnptools {type(pysnptools)}")

✓ xarray 2025.7.1
✓ pysnptools <class 'module'>


In [36]:
import numpy as np
# Get sample BED file using PySnpTools
from pysnptools.snpreader import Bed
from pysnptools.util import example_file

bed_file = example_file("tests/datasets/all_chr.maf0.001.N300.*")
snp_reader = Bed(bed_file, count_A1=True)
print(f"Shape: {snp_reader.shape} (individuals × SNPs)")

Shape: (300, 1015) (individuals × SNPs)


In [40]:
# Read all genotype data from disk to in-memory
snp_data = snp_reader.read()
print(snp_data.val)

# Instead read every second individual and SNPs (variants) from 20 to 30.
subset = snp_reader[::2, 20:30]
print(f"Subset shape: {subset.shape} (individuals × SNPs)")

# List the first 5 individual (sample) ids, 
# the first 5 SNP (variant) ids, and every unique chromosome. Then, read every value in chromosome 5.
print(snp_reader.iid[:5])
print(snp_reader.sid[:5])
print(np.unique(snp_reader.pos[:,0]))
val3 = snp_reader[:,snp_reader.pos[:,0] == 5].read().val
print(val3.shape)

[[0. 0. 1. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 ...
 [0. 0. 0. ... 0. 2. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 2. 0.]]
Subset shape: (150, 10) (individuals × SNPs)
[['POP1' '0']
 ['POP1' '12']
 ['POP1' '44']
 ['POP1' '58']
 ['POP1' '65']]
['1_12' '1_34' '1_10' '1_35' '1_28']
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23.]
(300, 43)


In [50]:
# Create a new snp_reader that re-orders and subsets the individuals and that leaves out chromosome 5.
iids_from_pheno_file = [["POP1", "65"], ["POP1", "44"], ["POP1", "0"]]
new_file_reader = snp_reader[snp_reader.iid_to_index(iids_from_pheno_file),snp_reader.pos[:,0] != 5]
print(f"# of individuals that will be read: {new_file_reader.iid_count}")
print(f"Note: not chrom 5 {np.unique(new_file_reader.pos[:,0])}")
# read just this data
val4 = new_file_reader.read().val
print(val4.shape)


# of individuals that will be read: 3
Note: not chrom 5 [ 1.  2.  3.  4.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19.
 20. 21. 22. 23.]
(3, 972)


In [25]:
# Convert PySnpTools BED data to xarray Dataset with full genomic metadata
import numpy as np

# Create xarray Dataset using numeric indexes (avoids MultiIndex complexity and ensures Zarr compatibility)
xarray_form = xr.Dataset(
    {
        "genotypes": (["individual", "snp"], snp_reader.read().val)
    },
    coords={
        "individual": range(snp_reader.iid_count),
        "fid": (["individual"], [fid for fid, _iid in snp_reader.read().iid]),  # Family IDs
        "iid": (["individual"], [iid for _fid, iid in snp_reader.read().iid]),  # Individual IDs
        
        "snp": range(snp_reader.sid_count),
        "sid": (["snp"], snp_reader.sid),                                                 # SNP IDs
        "chromosome": (["snp"], np.nan_to_num(snp_reader.pos[:, 0], nan=0).astype(int)),  # Chromosome (NaN→0)
        "cm_position": (["snp"], snp_reader.pos[:, 1]),                                   # Genetic position
        "bp_position": (["snp"], np.nan_to_num(snp_reader.pos[:, 2], nan=0).astype(int)), # Physical position (NaN→0)
    },
    attrs={
        "description": "Genotype data from PySnpTools BED file",
        "encoding": "0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=missing",
        "source": bed_file,
    }
)

display(xarray_form)

<xarray.Dataset> Size: 3MB
Dimensions:      (individual: 300, snp: 1015)
Coordinates:
  * individual   (individual) int64 2kB 0 1 2 3 4 5 ... 294 295 296 297 298 299
    fid          (individual) <U4 5kB 'POP1' 'POP1' 'POP1' ... 'POP1' 'POP1'
    iid          (individual) <U4 5kB '0' '12' '44' ... '2987' '2998' '2999'
  * snp          (snp) int64 8kB 0 1 2 3 4 5 6 ... 1009 1010 1011 1012 1013 1014
    sid          (snp) <U10 41kB '1_12' '1_34' '1_10' ... '23_49' '23_2' '23_3'
    chromosome   (snp) int64 8kB 1 1 1 1 1 1 1 1 1 ... 23 23 23 23 23 23 23 23
    cm_position  (snp) float64 8kB 0.008008 0.02302 0.07007 ... 0.9479 0.995
    bp_position  (snp) int64 8kB 0 1 4 6 7 8 9 10 11 ... 44 46 47 48 49 50 51 52
Data variables:
    genotypes    (individual, snp) float64 2MB 0.0 0.0 1.0 0.0 ... 0.0 2.0 0.0
Attributes:
    description:  Genotype data from PySnpTools BED file
    encoding:     0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=mis...
    source:       C:\Users\carlk\AppData\Local\Temp/hashdown/072f91aaa72b5340...

In [26]:
# Import zarr explicitly and save to Zarr format
import zarr
import warnings
print(f"✓ zarr {zarr.__version__}")
warnings.filterwarnings('ignore', category=UserWarning, module='zarr') # Can ignore unicode warnings

# Save to Zarr format for efficient storage and access
zarr_path = "all_chr.maf0.001.N300.zarr"
xarray_form.to_zarr(zarr_path, mode='w')

# Verify by loading back lazily
zarr_form = xr.open_zarr(zarr_path)
display(zarr_form)


✓ zarr 3.0.10


<xarray.Dataset> Size: 2MB
Dimensions:      (individual: 300, snp: 1015)
Coordinates:
    chromosome   (snp) int64 8kB ...
    bp_position  (snp) int64 8kB ...
  * snp          (snp) int64 8kB 0 1 2 3 4 5 6 ... 1009 1010 1011 1012 1013 1014
  * individual   (individual) int64 2kB 0 1 2 3 4 5 ... 294 295 296 297 298 299
    fid          (individual) object 2kB ...
    sid          (snp) object 8kB ...
    iid          (individual) object 2kB ...
    cm_position  (snp) float64 8kB ...
Data variables:
    genotypes    (individual, snp) float64 2MB ...
Attributes:
    description:  Genotype data from PySnpTools BED file
    encoding:     0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=mis...
    source:       C:\Users\carlk\AppData\Local\Temp/hashdown/072f91aaa72b5340...

In [27]:
# read all genotype data from the zarr file
zarr_form.genotypes.values  # Accessing the genotype data directly

array([[0., 0., 1., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 2., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 2., 0.]], shape=(300, 1015))

In [28]:
# Read every second individual and SNPs (variants) from 20 to 30.
val2 = zarr_form.genotypes[::2, 20:30].values
val2.shape

(150, 10)

In [29]:
# List the first 5 individual (sample) ids, the first 5 SNP (variant) ids,
# and every unique chromosome. Then, read every value in chromosome 5.
print(zarr_form.iid[:5].values)
print(zarr_form.sid[:5].values)
print(np.unique(zarr_form.chromosome))
print(zarr_form.genotypes[:, zarr_form.chromosome == 5].values)

['0' '12' '44' '58' '65']
['1_12' '1_34' '1_10' '1_35' '1_28']
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[[0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [57]:
# Create a new zarr reader that re-orders and subsets the individuals and that leaves out chromosome 5.
iids_from_pheno_file = [["POP1", "65"], ["POP1", "44"], ["POP1", "0"]]
indices = [i for i, (f, iid) in enumerate(zip(zarr_form.fid.values, zarr_form.iid.values)) if [f, iid] in iids_from_pheno_file]

# Do subsetting in two steps: individuals first, then chromosomes
new_zarr = zarr_form.isel(individual=indices).where(zarr_form.chromosome != 5, drop=True)
print(f"# of individuals: {new_zarr.sizes['individual']}")
print(f"Chromosomes: {np.unique(new_zarr.chromosome)}")
display(new_zarr)

# Read just this data
val4_zarr = new_zarr.genotypes.values
print(f"Shape: {val4_zarr.shape}")

# of individuals: 3
Chromosomes: [ 1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


<xarray.Dataset> Size: 62kB
Dimensions:      (individual: 3, snp: 972)
Coordinates:
    chromosome   (snp) int64 8kB 1 1 1 1 1 1 1 1 1 ... 23 23 23 23 23 23 23 23
    bp_position  (snp) int64 8kB 0 1 4 6 7 8 9 10 11 ... 44 46 47 48 49 50 51 52
  * snp          (snp) int64 8kB 0 1 2 3 4 5 6 ... 1009 1010 1011 1012 1013 1014
  * individual   (individual) int64 24B 0 2 4
    fid          (individual) object 24B 'POP1' 'POP1' 'POP1'
    sid          (snp) object 8kB '1_12' '1_34' '1_10' ... '23_49' '23_2' '23_3'
    iid          (individual) object 24B '0' '44' '65'
    cm_position  (snp) float64 8kB 0.008008 0.02302 0.07007 ... 0.9479 0.995
Data variables:
    genotypes    (individual, snp) float64 23kB 0.0 0.0 1.0 0.0 ... 1.0 0.0 0.0
Attributes:
    description:  Genotype data from PySnpTools BED file
    encoding:     0=homozygous ref, 1=heterozygous, 2=homozygous alt, NaN=mis...
    source:       C:\Users\carlk\AppData\Local\Temp/hashdown/072f91aaa72b5340...

Shape: (3, 972)


In [30]:
# Look at https://sgkit-dev.github.io/sgkit/latest/getting_started.html#getting-started